In [278]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense


# **Objetivo**
En este programa se hará una red neuronal que reciba como input el coeficiente de absorción y como output tengamos radio, fracción volumen y espesor de la película

### Primero juntaremos todos los archivos en uno solo

In [12]:
files = [file for file in os.listdir('./parametros')]

param = pd.DataFrame()
for file in files:
  df = pd.read_csv('./parametros/'+file)
  param = pd.concat([param, df], axis= 0)

Imprimimos la base de datos creada anteriormente para corroborar que se haya ejecutado como deseábamos la línea anterior.

In [13]:
param

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
0,301,0.016685,1,10,100
1,302,0.016167,1,10,100
2,303,0.015703,1,10,100
3,304,0.015290,1,10,100
4,305,0.014916,1,10,100
...,...,...,...,...,...
19195,896,0.002255,1,9,120
19196,897,0.002257,1,9,120
19197,898,0.002258,1,9,120
19198,899,0.002260,1,9,120


### Revolvemos los datos
Esto es para que el entrenamiento de la red sea óptimo y aís poderle dar una mayor gama de datos.

In [14]:
from sklearn.model_selection import train_test_split
param = param.sample(frac=1, random_state=42).reset_index(drop=True)
param

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
0,753,0.001533,1,23,120
1,310,0.014692,3,10,120
2,625,0.002815,1,10,85
3,604,0.000572,1,9,120
4,373,0.003104,1,21,120
...,...,...,...,...,...
38395,566,0.000711,1,10,70
38396,785,0.000418,12,10,120
38397,659,0.000772,1,9,120
38398,561,0.005686,1,10,105


## Separamos nuestra base de datos revuelta en dos
Una que servirá para entrenar el modelo y otra para comprobar que el modelo tenga un funcionamiento optimo. Esta división será de 80-20 respectivamente.

In [15]:
train_df, test_df = train_test_split(param, test_size=0.2, random_state=42)
#test será el 20% de los datos mientras train el 80%
test_df

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
27452,492,0.006226,5,10,120
15009,831,0.001993,1,26,120
8124,700,0.000512,1,26,120
38103,748,0.000418,11,10,120
25872,432,0.002877,1,10,65
...,...,...,...,...,...
3746,820,0.002003,7,10,120
18195,422,0.000481,1,7,120
37286,551,0.033743,4,10,120
15361,703,0.000239,1,29,120


In [16]:
train_df

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
36772,825,0.001564,11,10,120
22714,538,0.004552,1,10,65
3031,442,0.000189,1,10,120
9632,391,0.000021,1,10,115
29786,870,0.002197,1,11,120
...,...,...,...,...,...
6265,760,0.002772,1,10,100
11284,623,0.001487,1,32,120
38158,836,0.001522,1,10,135
860,324,0.008735,1,6,120


# Comenzamos a hacer la red neuronal


In [17]:
# Cargamos el DataFrame de entrenamiento
df = train_df
# Separarmos los datos de entrada y salida
X = df['Índice de absorción']
Y = df[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]

In [18]:
# Convertimos los dataframes en numpy arrays, esto para que su manejo sea más sencillo
X = np.array(X).reshape(-1, 1)
y = np.array(Y)

In [19]:
# Creamos el modelo de la red neuronal
model = Sequential()
model.add(Dense(64, input_dim=1, activation='relu'))
model.add(Dense(3, activation='linear'))

In [20]:
# Asignamos el tipo de error y el optimizador que vamos a utilizar
model.compile(loss='mean_absolute_error', optimizer='adam')

In [21]:
# Entrenamos a la red neuronal, en este caso lo hicimos en 100 épocas
model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
768/768 [==============================] - 1s 903us/step - loss: 39.1601 - val_loss: 32.3099
Epoch 2/100
768/768 [==============================] - 1s 730us/step - loss: 20.6608 - val_loss: 8.4786
Epoch 3/100
768/768 [==============================] - 1s 751us/step - loss: 5.1729 - val_loss: 4.8347
Epoch 4/100
768/768 [==============================] - 1s 744us/step - loss: 4.7821 - val_loss: 4.7919
Epoch 5/100
768/768 [==============================] - 1s 738us/step - loss: 4.7435 - val_loss: 4.7607
Epoch 6/100
768/768 [==============================] - 1s 769us/step - loss: 4.7284 - val_loss: 4.7542
Epoch 7/100
768/768 [==============================] - 1s 726us/step - loss: 4.7274 - val_loss: 4.7561
Epoch 8/100
768/768 [==============================] - 1s 728us/step - loss: 4.7268 - val_loss: 4.7545
Epoch 9/100
768/768 [==============================] - 1s 717us/step - loss: 4.7265 - val_loss: 4.7539
Epoch 10/100
768/768 [==============================] - 1s 711us/step 

In [22]:
# Hacemos las predicciones y las imprimimos
arr = test_df['Índice de absorción'].to_numpy()
y_pred = model.predict(arr)
print(y_pred)

240/240 [==============================] - 0s 510us/step
[[  1.0156828   9.996943  119.99479  ]
 [  1.0022719   9.998198  119.995056 ]
 [  0.9975815   9.998636  119.995155 ]
 ...
 [  1.1028519   9.988789  119.993034 ]
 [  0.9967154   9.998718  119.995186 ]
 [  1.0157286   9.996939  119.99479  ]]


In [23]:
# Almacenamos estas predicciones en un nuevo DataFrame
predicciones = pd.DataFrame(y_pred)
predicciones

,0,1,2
0,1.015683,9.996943,119.994789
1,1.002272,9.998198,119.995056
2,0.997581,9.998636,119.995155
3,0.997284,9.998665,119.995171
4,1.005074,9.997936,119.995010
...,...,...,...
7675,1.002305,9.998196,119.995064
7676,0.997483,9.998646,119.995163
7677,1.102852,9.988789,119.993034
7678,0.996715,9.998718,119.995186


In [24]:
# Renombramos las columnas para que equipare a la sintaxis de las bases de datos anteriores
predicciones.columns = ['Fracción Volumen', 'Radio (nm)', 'Espesor de la película (nm)']
predicciones

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,1.015683,9.996943,119.994789
1,1.002272,9.998198,119.995056
2,0.997581,9.998636,119.995155
3,0.997284,9.998665,119.995171
4,1.005074,9.997936,119.995010
...,...,...,...
7675,1.002305,9.998196,119.995064
7676,0.997483,9.998646,119.995163
7677,1.102852,9.988789,119.993034
7678,0.996715,9.998718,119.995186


### Redondeando los valores

In [25]:
redondeo_arriba = predicciones.round()
redondeo_arriba

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,1.0,10.0,120.0
1,1.0,10.0,120.0
2,1.0,10.0,120.0
3,1.0,10.0,120.0
4,1.0,10.0,120.0
...,...,...,...
7675,1.0,10.0,120.0
7676,1.0,10.0,120.0
7677,1.0,10.0,120.0
7678,1.0,10.0,120.0


In [26]:
redondeo_ceil =  predicciones.applymap(lambda x: math.ceil(x))
redondeo_ceil = redondeo_ceil.astype(float)
redondeo_ceil

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,2.0,10.0,120.0
1,2.0,10.0,120.0
2,1.0,10.0,120.0
3,1.0,10.0,120.0
4,2.0,10.0,120.0
...,...,...,...
7675,2.0,10.0,120.0
7676,1.0,10.0,120.0
7677,2.0,10.0,120.0
7678,1.0,10.0,120.0


In [27]:
redondeo_abajo = predicciones.apply(np.floor)
redondeo_abajo

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,1.0,9.0,119.0
1,1.0,9.0,119.0
2,0.0,9.0,119.0
3,0.0,9.0,119.0
4,1.0,9.0,119.0
...,...,...,...
7675,1.0,9.0,119.0
7676,0.0,9.0,119.0
7677,1.0,9.0,119.0
7678,0.0,9.0,119.0


# Probando la precisión del modelo
Primero lo haremos por cada parámetro. Se le restará los valores predichos por la red a los valores originales del dataframe test. Se contarán el número de ceros y ese será lo acertada que es nuestra red. De tener valores inferiores a 80% en número de ceros este rango se incrementará hasta llegar a un valor que el 80% de los valores sean menores a este.

## Fracción de Volumen

Calculando con el redondeo hacia abajo

In [28]:
redondeo_abajo_fracvol = redondeo_abajo['Fracción Volumen'].to_numpy()
test_df_fracvol = test_df['Fracvol'].to_numpy()
dif = redondeo_abajo_fracvol - test_df_fracvol
# count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en fracción volumen")

presición del  47.434895833333336 %. Con el redondeo para abajo en fracción volumen


In [29]:
num_zeros # Checamos el número de ceros que tenemos

3643

Umbral de 1

In [30]:
# Quiere decir que el 79.60% de los datos son (valor absoluto) 1 o 0 después de restar el test con el predicciones
umbral = 1
num = np.count_nonzero(np.abs(dif) <= umbral)
porcentaje = num*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en fracción volumen y un rango de 1")

presición del  77.39583333333333 %. Con el redondeo para abajo en fracción volumen y un rango de 1


In [31]:
nums = []
for num in dif:
    if np.abs(num) <= 1:
        nums.append(num)
len(nums)

5944

Calculando con el redondeo hacia arriba

In [32]:
redondeo_arriba_fracvol = redondeo_arriba['Fracción Volumen'].to_numpy()
test_df_fracvol = test_df['Fracvol'].to_numpy()
dif = redondeo_arriba_fracvol - test_df_fracvol
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para arriba en fracción volumen")

presición del  76.30208333333333 %. Con el redondeo para arriba en fracción volumen


In [33]:
num_zeros # Checamos el número de ceros que tenemos

5860

In [34]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en fracción volumen y un rango de", umbral)
    i = i+1

presición del  77.95572916666667 %. Con el redondeo para arriba en fracción volumen y un rango de 1
presición del  79.55729166666667 %. Con el redondeo para arriba en fracción volumen y un rango de 2
presición del  81.22395833333333 %. Con el redondeo para arriba en fracción volumen y un rango de 3


Calculando con celi

In [35]:
redondeo_ceil_fracvol = redondeo_ceil['Fracción Volumen'].to_numpy()
test_df_fracvol = test_df['Fracvol'].to_numpy()
dif = redondeo_ceil_fracvol - test_df_fracvol
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo con ceil en fracción volumen")

presición del  29.973958333333332 %. Con el redondeo con ceil en fracción volumen


In [36]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo ceil en fracción volumen y un rango de", umbral)
    i = i+1

presición del  79.07552083333333 %. Con el redondeo ceil en fracción volumen y un rango de 1
presición del  80.71614583333333 %. Con el redondeo ceil en fracción volumen y un rango de 2


# Radio

Calculando con el redondeo hacia abajo

In [37]:
redondeo_abajo_radio = redondeo_abajo['Radio (nm)'].to_numpy()
test_df_radio = test_df['Radio (nm)'].to_numpy()
dif = redondeo_abajo_radio - test_df_radio
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en radio")

presición del  1.3932291666666667 %. Con el redondeo para abajo en radio


In [38]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para abajo en radio y un rango de", umbral)
    i = i+1

presición del  55.2734375 %. Con el redondeo para abajo en radio y un rango de 1
presición del  58.515625 %. Con el redondeo para abajo en radio y un rango de 2
presición del  61.419270833333336 %. Con el redondeo para abajo en radio y un rango de 3
presición del  64.55729166666667 %. Con el redondeo para abajo en radio y un rango de 4
presición del  66.171875 %. Con el redondeo para abajo en radio y un rango de 5
presición del  67.8515625 %. Con el redondeo para abajo en radio y un rango de 6
presición del  69.58333333333333 %. Con el redondeo para abajo en radio y un rango de 7
presición del  71.2109375 %. Con el redondeo para abajo en radio y un rango de 8
presición del  72.87760416666667 %. Con el redondeo para abajo en radio y un rango de 9
presición del  74.27083333333333 %. Con el redondeo para abajo en radio y un rango de 10
presición del  75.6640625 %. Con el redondeo para abajo en radio y un rango de 11
presición del  77.27864583333333 %. Con el redondeo para abajo en radio y

Calculando con el redondeo hacia arriba

In [39]:
redondeo_arriba_radio = redondeo_arriba['Radio (nm)'].to_numpy()
test_df_radio = test_df['Radio (nm)'].to_numpy()
dif = redondeo_arriba_radio - test_df_radio
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para arriba en radio")

presición del  52.265625 %. Con el redondeo para arriba en radio


In [40]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en radio y un rango de", umbral)
    i = i+1

presición del  55.234375 %. Con el redondeo para arriba en radio y un rango de 1
presición del  58.138020833333336 %. Con el redondeo para arriba en radio y un rango de 2
presición del  61.393229166666664 %. Con el redondeo para arriba en radio y un rango de 3
presición del  64.62239583333333 %. Con el redondeo para arriba en radio y un rango de 4
presición del  67.8515625 %. Con el redondeo para arriba en radio y un rango de 5
presición del  69.58333333333333 %. Con el redondeo para arriba en radio y un rango de 6
presición del  71.2109375 %. Con el redondeo para arriba en radio y un rango de 7
presición del  72.87760416666667 %. Con el redondeo para arriba en radio y un rango de 8
presición del  74.27083333333333 %. Con el redondeo para arriba en radio y un rango de 9
presición del  75.6640625 %. Con el redondeo para arriba en radio y un rango de 10
presición del  77.27864583333333 %. Con el redondeo para arriba en radio y un rango de 11
presición del  78.90625 %. Con el redondeo par

Calculando con celi

In [41]:
redondeo_ceil_radio = redondeo_ceil['Radio (nm)'].to_numpy()
test_df_radio = test_df['Radio (nm)'].to_numpy()
dif = redondeo_ceil_radio - test_df_fracvol
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo el ceil en radio")

presición del  1.6145833333333333 %. Con el redondeo el ceil en radio


In [42]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en radio y un rango de", umbral)
    i = i+1

presición del  5.182291666666667 %. Con el redondeo para arriba en radio y un rango de 1
presición del  8.294270833333334 %. Con el redondeo para arriba en radio y un rango de 2
presición del  11.432291666666666 %. Con el redondeo para arriba en radio y un rango de 3
presición del  14.322916666666666 %. Con el redondeo para arriba en radio y un rango de 4
presición del  17.174479166666668 %. Con el redondeo para arriba en radio y un rango de 5
presición del  20.442708333333332 %. Con el redondeo para arriba en radio y un rango de 6
presición del  22.044270833333332 %. Con el redondeo para arriba en radio y un rango de 7
presición del  23.697916666666668 %. Con el redondeo para arriba en radio y un rango de 8
presición del  100.0 %. Con el redondeo para arriba en radio y un rango de 9


# Espesor

Calculando con el redondeo hacia abajo

In [43]:
redondeo_abajo_espesor = redondeo_abajo['Espesor de la película (nm)'].to_numpy()
test_df_espesor = test_df['Espesor de la película (nm)'].to_numpy()
dif = redondeo_abajo_espesor - test_df_espesor
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en espesor")

presición del  0.0 %. Con el redondeo para abajo en espesor


In [44]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para abajo en espesor y un rango de", umbral)
    i = i+1

presición del  76.10677083333333 %. Con el redondeo para abajo en espesor y un rango de 1
presición del  76.10677083333333 %. Con el redondeo para abajo en espesor y un rango de 2
presición del  76.10677083333333 %. Con el redondeo para abajo en espesor y un rango de 3
presición del  77.56510416666667 %. Con el redondeo para abajo en espesor y un rango de 4
presición del  77.56510416666667 %. Con el redondeo para abajo en espesor y un rango de 5
presición del  79.2578125 %. Con el redondeo para abajo en espesor y un rango de 6
presición del  79.2578125 %. Con el redondeo para abajo en espesor y un rango de 7
presición del  79.2578125 %. Con el redondeo para abajo en espesor y un rango de 8
presición del  80.88541666666667 %. Con el redondeo para abajo en espesor y un rango de 9


Calculando con el redondeo hacia arriba

In [45]:
redondeo_arriba_espesor = redondeo_arriba['Espesor de la película (nm)'].to_numpy()
test_df_espesor = test_df['Espesor de la película (nm)'].to_numpy()
dif = redondeo_arriba_espesor - test_df_espesor
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en espesor")

presición del  76.10677083333333 %. Con el redondeo para abajo en espesor


In [46]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en espesor y un rango de", umbral)
    i = i+1

presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 1
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 2
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 3
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 4
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 5
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 6
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 7
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 8
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 9
presición del  82.60416666666667 %. Con el redondeo para arriba en espesor y un rango de 10


Calculando con celi

In [47]:
redondeo_ceil_espesor = redondeo_ceil['Espesor de la película (nm)'].to_numpy()
test_df_espesor = test_df['Espesor de la película (nm)'].to_numpy()
dif = redondeo_ceil_espesor - test_df_espesor
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo ceil abajo en espesor")

presición del  76.10677083333333 %. Con el redondeo ceil abajo en espesor


In [48]:
# Se agrega umbral hasta llegar a 80% de aceptación
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en espesor y un rango de", umbral)
    i = i+1

presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 1
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 2
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 3
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 4
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 5
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 6
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 7
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 8
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 9
presición del  82.60416666666667 %. Con el redondeo para arriba en espesor y un rango de 10


# Colectivo

Ajustando la columna test para poder tenerla igual que las de redondeo

In [49]:
test2 = test_df.drop(['Longitud de onda', 'Índice de absorción'], axis=1)
test2 = test2.astype(float)
nuevo = {'Fracvol': 'Fracción Volumen'}
test2 = test2.rename(columns = nuevo)
test2 = test2.reset_index()
test2 = test2.drop(['index'], axis = 1)
test2

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,5.0,10.0,120.0
1,1.0,26.0,120.0
2,1.0,26.0,120.0
3,11.0,10.0,120.0
4,1.0,10.0,65.0
...,...,...,...
7675,7.0,10.0,120.0
7676,1.0,7.0,120.0
7677,4.0,10.0,120.0
7678,1.0,29.0,120.0


In [50]:
dif_abajo = test2 - redondeo_abajo
dif_abajo

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,4.0,1.0,1.0
1,0.0,17.0,1.0
2,1.0,17.0,1.0
3,11.0,1.0,1.0
4,0.0,1.0,-54.0
...,...,...,...
7675,6.0,1.0,1.0
7676,1.0,-2.0,1.0
7677,3.0,1.0,1.0
7678,1.0,20.0,1.0


In [51]:
nans3 = ~(dif_abajo == 0).all(axis=1)
df_sin_ceros = dif_abajo[nans3]
df_sin_ceros

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,4.0,1.0,1.0
1,0.0,17.0,1.0
2,1.0,17.0,1.0
3,11.0,1.0,1.0
4,0.0,1.0,-54.0
...,...,...,...
7675,6.0,1.0,1.0
7676,1.0,-2.0,1.0
7677,3.0,1.0,1.0
7678,1.0,20.0,1.0


In [52]:
df_sin_ceros.equals(dif_abajo)

True

Queremos que elimine los que tengan 2 0's

In [53]:
nans2 = dif_abajo
# Definimos una tolerancia
tolerancia = 1e-9
# Creamos una nueva columna que contenga el número de ceros por fila
nans2['zeros'] = (np.isclose(nans2, 0.0, atol=tolerancia)).sum(axis=1)
df_sin_ceros = nans2[nans2['zeros'] < 2]
df_sin_ceros = df_sin_ceros.drop(columns=['zeros'])
df_sin_ceros

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,4.0,1.0,1.0
1,0.0,17.0,1.0
2,1.0,17.0,1.0
3,11.0,1.0,1.0
4,0.0,1.0,-54.0
...,...,...,...
7675,6.0,1.0,1.0
7676,1.0,-2.0,1.0
7677,3.0,1.0,1.0
7678,1.0,20.0,1.0


In [54]:
nans2.equals(df_sin_ceros)

False

Queremos que elimine los que tenga un 0

In [55]:
dif_abajo = test2 - redondeo_abajo

In [56]:
nan1 = dif_abajo[(dif_abajo != 0).all(1)]
nan1

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,4.0,1.0,1.0
2,1.0,17.0,1.0
3,11.0,1.0,1.0
5,1.0,1.0,1.0
6,1.0,1.0,11.0
...,...,...,...
7674,1.0,16.0,1.0
7675,6.0,1.0,1.0
7676,1.0,-2.0,1.0
7677,3.0,1.0,1.0


In [57]:
df_sin_ceros.equals(dif_abajo)

False

# Redondeo hacia arriba

In [58]:
dif_arriba = redondeo_arriba - test2
dif_arriba

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,-4.0,0.0,0.0
1,0.0,-16.0,0.0
2,0.0,-16.0,0.0
3,-10.0,0.0,0.0
4,0.0,0.0,55.0
...,...,...,...
7675,-6.0,0.0,0.0
7676,0.0,3.0,0.0
7677,-3.0,0.0,0.0
7678,0.0,-19.0,0.0


Queremos que elimine los renglones que tengan 3 0's

In [59]:
nans3 = ~(dif_arriba == 0).all(axis=1)
df_sin_ceros = dif_arriba[nans3]
df_sin_ceros

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,-4.0,0.0,0.0
1,0.0,-16.0,0.0
2,0.0,-16.0,0.0
3,-10.0,0.0,0.0
4,0.0,0.0,55.0
...,...,...,...
7675,-6.0,0.0,0.0
7676,0.0,3.0,0.0
7677,-3.0,0.0,0.0
7678,0.0,-19.0,0.0


In [60]:
df_sin_ceros.equals(dif_arriba)

False

Queremos que ensenie los que tengan 2 0's

In [77]:
#Se imprimen las columnas que tengan solo 2 ceros
nans2 = dif_arriba
# Definimos una tolerancia
tolerancia = 1e-9
# Creamos una nueva columna que contenga el número de ceros por fila
nans2['zeros'] = (np.isclose(nans2, 0.0, atol=tolerancia)).sum(axis=1)
df_sin_ceros = nans2[nans2['zeros'] == 2]
df_sin_ceros = df_sin_ceros.drop(columns=['zeros'])
df_sin_ceros

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,-4.0,0.0,0.0
1,0.0,-16.0,0.0
2,0.0,-16.0,0.0
3,-10.0,0.0,0.0
4,0.0,0.0,55.0
...,...,...,...
7675,-6.0,0.0,0.0
7676,0.0,3.0,0.0
7677,-3.0,0.0,0.0
7678,0.0,-19.0,0.0


Queremos que elimine los que tenga un NaN

In [78]:
dif_abajo = test2 - redondeo_abajo

In [79]:
nan1 = dif_arriba[(dif_arriba != 0).all(1)]
nan1

,Fracción Volumen,Radio (nm),Espesor de la película (nm),zeros


In [80]:
def evaluar_red_neuronal(y_pred, y_true):
    # Convertir los DataFrames a numpy arrays
    y_pred = y_pred.values
    y_true = y_true.values
    
    # Calcular el MSE y el R2
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Imprimir los resultados
    print("MSE: ", mse)
    print("R2: ", r2)
    
    # Devolver el MSE y el R2 como una tupla
    return (mse, r2)

In [81]:
mse, r2 = evaluar_red_neuronal(redondeo_abajo['Espesor de la película (nm)'], test2['Espesor de la película (nm)'])


MSE:  249.80208333333334
R2:  -0.08938671118283525


In [82]:
mse, r2 = evaluar_red_neuronal(redondeo_abajo['Fracción Volumen'], test2['Fracción Volumen'])


MSE:  20.027734375
R2:  -0.2886903689715221


In [83]:
mse, r2 = evaluar_red_neuronal(redondeo_abajo['Radio (nm)'], test2['Radio (nm)'])


MSE:  106.77421875
R2:  -0.5408232160122908


# Ahora invertimos la red, de input será Fracción vol, espesor y radio. Output longitud de onda e Índice de absorción

In [244]:
# Cargar DataFrame
df = train_df
# Separar datos de entrada y salida
Y = df[['Índice de absorción', 'Longitud de onda']]
X = df[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]

In [245]:
# Convertimos los dataframes en numpy arrays
X = np.array(X)
y = np.array(Y)

In [312]:
# Creamos el modelo de la red neuronal
model = Sequential()
model.add(Dense(64, input_dim=3, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='linear'))

In [313]:
model.compile(loss='mean_absolute_error', optimizer='adam')

In [314]:
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
960/960 [==============================] - 1s 597us/step - loss: 298.8497
Epoch 2/10
960/960 [==============================] - 1s 596us/step - loss: 296.4446
Epoch 3/10
960/960 [==============================] - 1s 583us/step - loss: 294.0446
Epoch 4/10
960/960 [==============================] - 1s 581us/step - loss: 291.6448
Epoch 5/10
960/960 [==============================] - 1s 582us/step - loss: 289.2451
Epoch 6/10
960/960 [==============================] - 1s 587us/step - loss: 286.8454
Epoch 7/10
960/960 [==============================] - 1s 582us/step - loss: 284.4458
Epoch 8/10
960/960 [==============================] - 1s 579us/step - loss: 282.0455
Epoch 9/10
960/960 [==============================] - 1s 582us/step - loss: 279.6451
Epoch 10/10
960/960 [==============================] - 1s 578us/step - loss: 277.2444


In [315]:
arr = test_df[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]

In [316]:
y_pred = model.predict(arr)
print(y_pred)

240/240 [==============================] - 0s 524us/step
[[4.3750005e-03 4.7982609e+01]
 [4.3750005e-03 4.7982609e+01]
 [4.3750005e-03 4.7982609e+01]
 ...
 [4.3750005e-03 4.7982609e+01]
 [4.3750005e-03 4.7982609e+01]
 [4.3750005e-03 4.7982609e+01]]


In [317]:
predicciones = pd.DataFrame(y_pred)
predicciones

,0,1
0,0.004375,47.982609
1,0.004375,47.982609
2,0.004375,47.982609
3,0.004375,47.982609
4,0.004375,47.982609
...,...,...
7675,0.004375,47.982609
7676,0.004375,47.982609
7677,0.004375,47.982609
7678,0.004375,47.982609


# Renombrando las columnas

In [318]:
predicciones.columns = ['Índice de absorción', 'Longitud de onda']
predicciones

,Índice de absorción,Longitud de onda
0,0.004375,47.982609
1,0.004375,47.982609
2,0.004375,47.982609
3,0.004375,47.982609
4,0.004375,47.982609
...,...,...
7675,0.004375,47.982609
7676,0.004375,47.982609
7677,0.004375,47.982609
7678,0.004375,47.982609


# Redondeos

In [319]:
redondeo_arriba = predicciones.copy()
redondeo_arriba['Longitud de onda'] = redondeo_arriba['Longitud de onda'].round()
redondeo_arriba

,Índice de absorción,Longitud de onda
0,0.004375,48.0
1,0.004375,48.0
2,0.004375,48.0
3,0.004375,48.0
4,0.004375,48.0
...,...,...
7675,0.004375,48.0
7676,0.004375,48.0
7677,0.004375,48.0
7678,0.004375,48.0


In [320]:
redondeo_ceil = predicciones.copy()
redondeo_ceil['Longitud de onda'] = np.ceil(redondeo_ceil['Longitud de onda'])
redondeo_ceil

,Índice de absorción,Longitud de onda
0,0.004375,48.0
1,0.004375,48.0
2,0.004375,48.0
3,0.004375,48.0
4,0.004375,48.0
...,...,...
7675,0.004375,48.0
7676,0.004375,48.0
7677,0.004375,48.0
7678,0.004375,48.0


In [321]:
redondeo_abajo = predicciones.copy()
redondeo_abajo['Longitud de onda'] = np.floor(redondeo_abajo['Longitud de onda'])
redondeo_abajo

,Índice de absorción,Longitud de onda
0,0.004375,47.0
1,0.004375,47.0
2,0.004375,47.0
3,0.004375,47.0
4,0.004375,47.0
...,...,...
7675,0.004375,47.0
7676,0.004375,47.0
7677,0.004375,47.0
7678,0.004375,47.0


# Probando la accurancy del modelo

Primero lo haremos por cada parámetro. Se le restará los valores predichos por la red a los valores originales de test. Se contarán el número de ceros y ese será lo acertada que es nuestra red. De tener valores inferiores a 80% en número de ceros este rango se incrementará hasta llegar a un valor que el 80% de los valores sean menores a este

# Longitud de onda

Redondeo abajo

In [308]:
redondeo_abajo_long = redondeo_abajo['Longitud de onda'].to_numpy()
test_df_long = test_df['Longitud de onda'].to_numpy()
dif = redondeo_abajo_long - test_df_long 
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en longitud de onda")

presición del  0.13020833333333334 %. Con el redondeo para abajo en longitud de onda


Redondeo ceil

In [309]:
redondeo_ceil_long = redondeo_ceil['Longitud de onda'].to_numpy()
test_df_long = test_df['Longitud de onda'].to_numpy()
dif = redondeo_ceil_long - test_df_long
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo ceil en longitud de onda")

presición del  0.1953125 %. Con el redondeo ceil en longitud de onda


Redondeo hacia arriba

In [323]:
redondeo_arriba_long = redondeo_arriba['Longitud de onda'].to_numpy()
test_df_long = test_df['Longitud de onda'].to_numpy()
dif = redondeo_arriba_long - test_df_long
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo arriba en longitud de onda")

presición del  0.0 %. Con el redondeo arriba en longitud de onda


# Índice de absorción

In [322]:
redondeo_abajo_IA = redondeo_abajo['Índice de absorción'].to_numpy()
test_df_IA = test_df['Índice de absorción'].to_numpy()
dif = redondeo_abajo_IA - test_df_IA 
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en Índice de absorción")

presición del  0.0 %. Con el redondeo para abajo en Índice de absorción


In [ ]:
# Define la entrada
input_layer = Input(shape=(2,))

# Capas ocultas compartidas
hidden1 = Dense(64, activation='relu')(input_layer)
hidden2 = Dense(32, activation='relu')(hidden1)

# Ramas para cada salida
branch1 = Dense(16, activation='relu')(hidden2)
output1 = Dense(1, activation='linear', name='fraccion_volumen')(branch1)

branch2 = Dense(16, activation='relu')(hidden2)
output2 = Dense(1, activation='linear', name='radio')(branch2)

branch3 = Dense(16, activation='relu')(hidden2)
output3 = Dense(1, activation='linear', name='espesor')(branch3)

# Define el modelo
model = Model(inputs=input_layer, outputs=[output1, output2, output3])

# Compila el modelo
model.compile(loss='mean_squared_error', optimizer='adam')